#  Amazon Bedrock Activation Workshop Chapter2: 検索拡張生成(RAG)

RAG (Retrieval Augmented Generation, 検索拡張生成)とは、DB・検索インデックス等の信頼できる外部ソースから取得した情報を Prompt に含めることによって、ハルシネーション(幻覚)を抑止するというプロンプトエンジニアリング手法の一種です。  

接続する外部ソースに制限はなく、Amazon Kendra のようなエンタープライズサーチシステムや OpenSearch, pinecone といったベクターDB などが一般的に使われます。

この Chapter では、RAG の理解を深めていただくことを目的として、埋め込み表現モデルを使った簡易的なベクトル検索システムを作り、LLM と組み合わせることで RAG システムを実装します。  
（実際に外部ソースには接続しません）

## 準備


In [ ]:
import boto3
import json
import numpy as np
bedrock_runtime = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

In [ ]:
def invoke_claude(text, max_tokens_to_sample=1000): 

    body = json.dumps({
        "prompt": f"\n\nHuman:{text}\n\nAssistant:",
        "max_tokens_to_sample": max_tokens_to_sample,
        "temperature": 0.1,
        "top_p": 0.9,
    })

    modelId = 'anthropic.claude-v2'
    accept = 'application/json'
    contentType = 'application/json'

    response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

    response_body = json.loads(response.get('body').read())

    # text
    # print(response_body.get('completion'))
    return response_body.get('completion')

## ベクトル検索システムの作成

今回は、次に挙げるようなAmazon デバイスのリストを検索対象のサンプルとして使います。  
実際にはこれらのデータが DB、検索インデックスに格納されていると想定してください。

In [ ]:
# 検索対象として、Amazonデバイスのリストを使います
products = [
    {
        "name": "Fire TV Stick 第3世代 | HD対応スタンダードモデル |ストリーミングメディアプレイヤー【2020年発売】",
        "description": """人気のFire TV Stickが第2世代のモデルよりも50%パワフルになりました。フルHDの動画をすばやくストリーミングでき、HDR、Dolby Atmosにも対応しています。(対応するコンテンツや機器が必要です)
付属のリモコンではAlexaに話しかけて音声でコンテンツを検索・再生操作できます。お気に入りのコンテンツに簡単にアクセスできるアプリボタンと番組表ボタンが追加されました。対応するテレビ・サウンドバーの電源、ボリュームもコントロールできます。
Prime Video、YouTube、Netflix、TVer、U-NEXT、DAZN、Disney+、FOD、Apple TV+などの豊富な映画やビデオを大画面で楽しめる。Silk BrowserによりFacebook、Twitterなど様々なウェブサイトにもアクセス可能。
さらにプライム会員なら、Prime Videoの会員特典対象の作品が追加料金なしで見放題。映画、ドラマ、アニメ、お笑い・バラエティ番組など充実のコンテンツ。また、Amazon Music Primeで1億曲がシャッフル再生で聴き放題。
Prime Videoチャンネル、ABEMA、Hulu、DAZN、Redbull TVなどのニュース、スポーツ、バラエティ、ドラマ、将棋など様々なジャンルのライブ配信コンテンツが見られます。
Amazon Music、Spotifyなどからお好みの曲をストリーミング再生します。*サービスの利用には別途登録・契約や料金が必要な場合があります。
簡単セットアップ。お持ちのテレビのHDMI端子に挿してwifiにつなぐだけ。"""
    },
    {
        "name": "Echo (エコー) 第4世代 - スマートスピーカーwith Alexa",
        "description": """
        【一新されたデザインとサウンド】クリアな高音、ダイナミックな中音、そして深みのある低音で、リッチで細やかなサウンドを、設置場所に合わせてパワフルなスピーカーがお届け。
【声で音楽をリクエスト】Amazon Music、Apple Music、Spotifyなどからお好みの曲をストリーミング再生。ラジオ局やAudibleのオーディオブックも。
【Alexaにおまかせ】ニュースや天気予報を聞いたり、タイマーやアラームを設定したり、対応するスマートデバイスを操作したり、いろいろな質問をしたり。Alexaがさまざまなことをお手伝い。
【かんたんスマートホーム】内蔵ハブでZigbee対応スマートデバイスの設定が簡単。
【サウンドで満たそう】違う部屋に設置した複数のEchoデバイスで同じ音楽を同時に再生できます。Fire TVと組み合わせて、臨場感のあるエンターテイメントを楽しむことも。
【家族や友人とつながる】Echoデバイスを使っている友人とハンズフリーで通話したり、他の部屋に置いたEchoデバイスに呼びかけたり、家中にアナウンスしたりも。
【プライバシーに配慮したデザイン】マイクの電源を切ることができるマイク オン/オフ ボタンを用意するなど、何重ものプライバシー保護対策を用いて設計しています。
        """
    },
    {
        "name": "Fire HD 8 タブレット - 8インチHD ディスプレイ 32GB ブラック (2022年発売)",
        "description": """
        【前世代機から最大30％高速化】2GB RAM、6コアプロセッサ搭載。
【最大13時間稼働バッテリー】同梱の5W USB-C (2.0)充電アダプタで、フル充電まで約5時間。
【コンテンツが充実】Prime Video、Netflix、ディズニープラス、U-NEXTなどでお好きな番組や映画をストリーミングやダウンロードで楽しめます。
【薄くて、軽くて、丈夫】前世代機より薄く軽く、強化アルミノシリケートグラス製のスクリーン。落下テストでの耐久性はApple iPad Mini (2021)の2倍。
【Alexa搭載】Alexaに話しかけて音楽を再生、天気やニュースの確認、Alexa対応スマートホームデバイス（別売り）の操作ができます。
【HDビデオ通話】AlexaアプリやZoomアプリをお持ちの友人や家族とビデオ通話が可能。
        """
    },
    {
        "name": "Kindle (16GB) 6インチディスプレイ 電子書籍リーダー ブラック",
        "description": """
        より軽く、コンパクトになったKindle。300ppiの高解像度ディスプレイで、文字と画像をくっきり表示。
光の反射を抑えた、紙のような読み心地。明るさ調節可能なフロントライトやダークモード搭載で、いつでも快適に読書できます。
本に夢中になれる贅沢を。Eメールやソーシャルメディアなどの通知に気を取られることなく、本に集中できる読書のための専用端末。
長時間持続するバッテリー。USB-Cケーブルによる1度のフル充電で、最大6週間読書を楽しめます。
前モデルの2倍の16GBのストレージ。この1台に数千冊を保存できます（一般的な書籍の場合）。
Kindle Unlimitedに会員登録すれば、200万冊以上の本・マンガ・雑誌・洋書が読み放題。新しいお気に入りがきっと見つかります。
サステナビリティに配慮。Kindle本体には再生利用素材が使用されています。
        """
    }
    
]

### 埋め込み表現モデルの活用
まずはAmazon Titan Embeddingモデルを使い、文字列の埋め込み表現を取得する関数を定義します

In [ ]:
# 入力文字列の埋め込み表現を取得する関数
def get_embedding(text: str) -> np.ndarray:
    body = json.dumps({
    "inputText": text
    })

    modelId =  "amazon.titan-embed-text-v1"
    accept = '*/*'
    contentType = 'application/json'

    response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    # print(response)
    response_embeddings = np.array(response_body.get('embedding'))
    
    return response_embeddings

2つの文字列間の類似度を計算するため、埋め込み表現のベクトルどうしで、コサイン類似度を計算するような関数を定義します。

In [ ]:
# 2つのベクトルのコサイン類似度を計算する関数
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
print(cos_sim(get_embedding('富士山は日本で一番高い山です'), get_embedding('日本最高峰はなんですか')))
print(cos_sim(get_embedding('富士山は日本で一番高い山です'), get_embedding('バナナはおやつに入りますか')))

実行結果から、関連性の高い文章の類似度が高くなることが確認できます

### 商品リストの埋め込み表現の作成

In [ ]:
products_embeddings = []
for ind, product in enumerate(products):
    embeddings = get_embedding(product['name'])
    products[ind]['embedding'] = embeddings

In [ ]:
print(products)

### 商品リストを検索する関数の作成

In [ ]:
from typing import List
# クエリの埋め込み表現を受け取り、商品リストの埋め込み表現の中から最も類似度が高いエントリを検索する関数
def search_embeddings(query_embedding: np.ndarray, arr: List, embd_name = 'embedding', threshold = 0.5) -> List:
    cos_sims = []
    
    # リストの中の埋め込み表現とのコサイン類似度を計算
    for embd in [e[embd_name] for e in arr]:
        cos_sims.append(cos_sim(embd, query_embedding))
    
    # 類似度の降順にソート
    decorated = [(cos_sims[i], i, element) for i, element in enumerate(arr)]
    decorated.sort(reverse=True)
    
    # 類似度が一定のしきい値を超えているエントリのみを返す
    result = [(cos_sim, element) for cos_sim, _, element in decorated if cos_sim > threshold]
    
    return result

In [ ]:
query = "電子書籍"
result = search_embeddings(get_embedding(query), products)
print('コサイン類似度: ', result[0][0])
print('名前: ', result[0][1]['name'])
print('説明: ', result[0][1]['description'])

In [ ]:
query = "動画配信"
result = search_embeddings(get_embedding(query), products)
print('コサイン類似度: ', result[0][0])
print('名前: ', result[0][1]['name'])
print('説明: ', result[0][1]['description'])

## RAG の作成

ベクトル検索エンジンと LLM を組み合わせて、検索結果をもとに質問に答えるような RAG システムを構築します。

In [ ]:
# LLMのタスク指示をinstructionで、ユーザからのクエリをquestionに、検索結果をcontextへ反映します。
prompt_template = \
"""
<instruction>
あなたは親切なAIボットです。ユーザからの質問に対してcontextで与えられている情報をもとに誠実に回答します。
ただし、質問に対する答えがcontextに書かれていない場合は、正直に「分かりません。」と回答してください。
</instruction>
<question>
{query}
</question>
<context>
{context}
</context>
"""

In [ ]:
query = "電子書籍に適した端末"
# query内容をもとにproductsを検索
search_result = search_embeddings(get_embedding(query), products)
context = ''
if len(search_result) > 0:
    context = 'name: '+search_result[0][1]['name'] + 'description:' + search_result[0][1]['description']

result = invoke_claude(prompt_template.format(query=query, context=context))

In [ ]:
print('検索結果:\n', context)
print('LLMによる回答:\n', result)

In [ ]:
query = "動画配信を楽しむには何を買えばいいですか"
# query内容をもとにproductsを検索
search_result = search_embeddings(get_embedding(query), products)
context = ''
if len(search_result) > 0:
    context = 'name: '+search_result[0][1]['name'] + 'description:' + search_result[0][1]['description']

result = invoke_claude(prompt_template.format(query=query, context=context))

In [ ]:
print('検索結果:\n', context)
print('LLMによる回答:\n', result)

In [ ]:
query = "私の父は読書が好きです。どういったプレゼントを贈るのが良いでしょうか"
# query内容をもとにproductsを検索
search_result = search_embeddings(get_embedding(query), products)
context = ''
if len(search_result) > 0:
    context = 'name: '+search_result[0][1]['name'] + 'description:' + search_result[0][1]['description']


result = invoke_claude(prompt_template.format(query=query, context=context))

In [ ]:
print('検索結果:\n', context)
print('LLMによる回答:\n', result)

以上のように、検索システムと LLM を組み合わせることで、情報源をもとにした信頼できる回答を生成することができます。

## RAG の性能評価

RAG による効果も測定する必要があります。  
RAG の評価方法として、様々な方式[1](https://www.databricks.com/blog/LLM-auto-eval-best-practices-RAG), [2](https://betterprogramming.pub/llamaindex-how-to-evaluate-your-rag-retrieval-augmented-generation-applications-2c83490f489)が提案されています。  
今回は、LLM による回答結果を 1:良かった、0:悪かった の2値で人によってフィードバックし、スコアリングします。

In [ ]:
test_query = [
    "電子書籍に適した端末",
    "動画配信を楽しむには何を買えばいいですか",
    "安価なタブレット端末が欲しいです。",
    "私の祖母は植物の動画が大好きで、いつもスマホでYouTubeを見ています。そんな彼女に贈り物を贈りたいのですが、何を贈ると喜んでもらえるでしょうか",
    "私には5歳になる娘がいます。彼女はPrime Videoが大好きで、いつもiPadで動画を見ています。そんな彼女に贈り物を贈りたいのですが、何がいいでしょうか"
]

In [ ]:
def rag_search(query: str) -> str:
    prompt_template = \
    """
    <instruction>
    あなたは親切なAIボットです。ユーザからの質問に対してcontextで与えられている情報をもとに誠実に回答します。
    ただし、質問に対する答えがcontextに書かれていない場合は、正直に「分かりません。」と回答してください。
    </instruction>
    <question>
    {query}
    </question>
    <context>
    {context}
    </context>
    """
    
    search_result = search_embeddings(get_embedding(query), products)
    context = ''
    if len(search_result) > 0:
        context = 'name: '+search_result[0][1]['name'] + 'description:' + search_result[0][1]['description']


    result = invoke_claude(prompt_template.format(query=query, context=context))
    return result

In [ ]:
user_feedback = []

for query in test_query:
    result = rag_search(query)
    print('query:\n', query)
    print('result:\n', result)
    print('\n以上の回答を 1:良かった、0:悪かった の いずれかでフィードバックしてください。:', end='')
    feedback = int(input())
    user_feedback.append(feedback)

In [ ]:
csat = sum(user_feedback) / len(user_feedback)
print('顧客満足度: ', csat)

このように、生成結果をフィードバックする仕組みがあると、RAGによる効果を測定することができます。  
得られたフィードバックをもとに、よりよい回答が出せるようなプロンプトや検索の仕組みを考えてみましょう

## RAG の改善

上記の RAG による回答を見ると、単純な質問であれば答えられるものの、シチュエーションを踏まえた複雑な質問になると検索精度が下がり回答が生成できていないことがわかります。  
そこで、クエリをいきなりベクトル検索するのではなく、まずは LLM でキーワードを抽出してから検索するような仕組みに変えてみましょう。

In [ ]:
prompt_template = \
"""
<instruction> 
あなたは親切なAIボットです。ユーザからの質問に対して正直に答えるために検索を行うことができます。
質問の内容をもとに、最適な検索クエリを出力してください。
ただし、返答作成時はexampleに書かれているような単語のリストで回答してください。
また、単語リスト以外の情報やコメントは含めないでください。
</instruction>
<question>{question}</question>
<example>
週末 旅行先 人気
</example>
"""

In [ ]:
prompt = prompt_template.format(question = test_query[4])
result = invoke_claude(prompt)
print('prompt:', prompt)
print('output:', result)

キーワードが抽出できていることがわかります。ではベクトル検索と組み合わせてみましょう

In [ ]:
def rag_search2(query: str) -> str:
    search_prompt_template = \
    """
    <instruction> 
    あなたは親切なAIボットです。ユーザからの質問に対して正直に答えるために検索を行うことができます。
    質問の内容をもとに、最適な検索クエリを出力してください。
    ただし、返答作成時はexampleに書かれているような単語のリストで回答してください。
    また、単語リスト以外の情報やコメントは含めないでください。
    </instruction>
    <question>{question}</question>
    <example>
    週末 旅行先 人気
    </example>
    """
    search_prompt = search_prompt_template.format(question=query)
    search_query = invoke_claude(search_prompt)
    
    print("keyword: ", search_query)
    
    
    search_result = search_embeddings(get_embedding(search_query), products)
    context = ''
    if len(search_result) > 0:
        context = 'name: '+search_result[0][1]['name'] + 'description:' + search_result[0][1]['description']

        
    prompt_template = \
    """
    <instruction>
    あなたは親切なAIボットです。ユーザからの質問に対してcontextで与えられている情報をもとに誠実に回答します。
    ただし、質問に対する答えがcontextに書かれていない場合は、正直に「分かりません。」と回答してください。
    </instruction>
    <question>
    {query}
    </question>
    <context>
    {context}
    </context>
    """
    result = invoke_claude(prompt_template.format(query=query, context=context))
    return result

In [ ]:
for query in test_query:
    print('query:', query)
    print('前者のRAG')
    print(rag_search(query))
    print('後者のRAG')
    print(rag_search2(query))

In [ ]:
print(rag_search2(test_query[4]))

いかがでしょうか、結果は改善されたでしょうか  
適宜プロンプトも変えて様々なセットを試して見てください。 　

RAG システムの改善には複数の要素が作用するため、適切な改善を行うためには、統一的な指標を定めておくことが重要です。
あらかじめ想定質問のセットを作っておくことは、RAG による効果を測定する上で効果的です。  

## まとめ

このチャプターではRAGの概要と基本的な考え方をご紹介しました。  
今回はベクトルサーチベースで実装しましたが、精度向上のためには、外部DBの選定も重要になります。  
ぜひ自分たちに最適な構成を一緒に見つけていきましょう。